---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv', encoding="latin-1")
train.head()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [3]:
test=pd.read_csv('test.csv')
test.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [4]:
#print(train.columns)
#print(test.columns)

In [5]:
#print(train.country.value_counts(dropna=False))
#print(test.country.value_counts(dropna=False))

In [6]:
#print(train.city.value_counts(dropna=False))
#print(test.city.value_counts(dropna=False))

In [7]:
#print(train.state.value_counts(dropna=False))
#print(test.state.value_counts(dropna=False))

In [8]:
#print(train.disposition.value_counts(dropna=False))
#print(test.disposition.value_counts(dropna=False))

In [9]:
#print(train.fine_amount.value_counts(dropna=False))
#print(test.fine_amount.value_counts(dropna=False))

In [10]:
#train.admin_fee.value_counts(dropna=False)

In [11]:
#train.state_fee.value_counts(dropna=False)

In [12]:
#train.late_fee.value_counts(dropna=False)

In [13]:
#train.clean_up_cost.value_counts(dropna=False)

In [14]:
#train.judgment_amount.value_counts(dropna=False)

In [15]:
#train.collection_status.value_counts(dropna=False)

In [16]:
#train.balance_due.value_counts(dropna=False)

In [17]:
#train.compliance_detail.value_counts(dropna=False)

In [18]:
#len(train.city.unique())

In [19]:
#train['compliance'].value_counts(dropna=False)

In [20]:
#print(train.violator_name.value_counts(dropna=False))
#print(test.violator_name.value_counts(dropna=False))
#train.violator_name.unique()

In [21]:
#print(train.agency_name.value_counts(dropna=False))
#print(test.agency_name.value_counts(dropna=False))

In [22]:
#print(train.inspector_name.value_counts(dropna=False))
#print(test.inspector_name.value_counts(dropna=False))

In [23]:
#print(train.violation_code.value_counts(dropna=False))
#print(test.violation_code.value_counts(dropna=False))

In [24]:
#train.violation_description.value_counts(dropna=False)

In [26]:
#col_list = ['ticket_id', 'agency_name', 'state', 'ticket_issued_date', 'hearing_date', 'violation_code', 'disposition', 'fine_amount', 'compliance']

In [27]:
#data selection & removing compliance missing value
'''
train1 = train[col_list]
test1 = test[col_list[:-1]]
train1 = train1[train1.compliance.notnull()]
train1.head()
'''

'\ntrain1 = train[col_list]\ntest1 = test[col_list[:-1]]\ntrain1 = train1[train1.compliance.notnull()]\ntrain1.head()\n'

In [28]:
#address = pd.read_csv('addresses.csv')
#address.head()

In [29]:
#latlons = pd.read_csv('latlons.csv')
#latlons.head()

In [30]:
#merge address and latlons data on 'address'
#add_lat = pd.merge(address, latlons, on ='address', how = 'left')
#add_lat.head()

In [31]:
#merge train and add_lat on 'ticket_id'
#train2 = pd.merge(train1, add_lat, on='ticket_id', how='left')
#train2.head()

In [32]:
#merge test and add_lat on 'ticket_id'
#test2 = pd.merge(test1, add_lat, on='ticket_id', how='left')
#test2.head()

In [33]:
#print(train2.address.value_counts(dropna=False))
#print(test2.address.value_counts(dropna=False))

In [34]:
#train1.set_index('ticket_id', inplace=True)
#test1.set_index('ticket_id', inplace=True)

In [35]:
#train1.compliance.value_counts(dropna=False)

In [36]:
#train1.fine_amount.value_counts(normalize = True, dropna=False)

In [37]:
'''
col_list = ['ticket_id', 'agency_name', 'state','disposition','fine_amount', 'compliance'] #'ticket_issued_date', 'hearing_date', 'disposition', 'fine_amount'
train3 = train[col_list]
test3 = test[col_list[:-1]]
train3 = train3[train3.compliance.notnull()]
train3.set_index('ticket_id', inplace=True)
test3.set_index('ticket_id', inplace=True)
'''

"\ncol_list = ['ticket_id', 'agency_name', 'state','disposition','fine_amount', 'compliance'] #'ticket_issued_date', 'hearing_date', 'disposition', 'fine_amount'\ntrain3 = train[col_list]\ntest3 = test[col_list[:-1]]\ntrain3 = train3[train3.compliance.notnull()]\ntrain3.set_index('ticket_id', inplace=True)\ntest3.set_index('ticket_id', inplace=True)\n"

In [38]:
'''
print(set(test3.agency_name) - set(train3.agency_name))
print(set(train3.agency_name) - set(test3.agency_name))
print(set(train3.agency_name) & set(test3.agency_name))
agency_name_list0 = set(train3.agency_name) & set(test3.agency_name)
agency_name_list = ['agency_name_'+ x for x in agency_name_list0]
agency_name_list
'''

"\nprint(set(test3.agency_name) - set(train3.agency_name))\nprint(set(train3.agency_name) - set(test3.agency_name))\nprint(set(train3.agency_name) & set(test3.agency_name))\nagency_name_list0 = set(train3.agency_name) & set(test3.agency_name)\nagency_name_list = ['agency_name_'+ x for x in agency_name_list0]\nagency_name_list\n"

In [39]:
'''
print(set(test3.disposition) - set(train3.disposition))
print(set(train3.disposition) - set(test3.disposition))
disposition_list0 = set(train3.disposition) & set(test3.disposition)
disposition_list = ['disposition_'+ x for x in disposition_list0]
disposition_list
'''

"\nprint(set(test3.disposition) - set(train3.disposition))\nprint(set(train3.disposition) - set(test3.disposition))\ndisposition_list0 = set(train3.disposition) & set(test3.disposition)\ndisposition_list = ['disposition_'+ x for x in disposition_list0]\ndisposition_list\n"

In [40]:
'''
print(set(test3.state) - set(train3.state))
print(set(train3.state) - set(test3.state))
print(set(train3.state) & set(test3.state))
state_list0 = set(train3.state) & set(test3.state)
state_list = ['state_'+ x for x in state_list0 if str(x) != 'nan']
state_list
'''

"\nprint(set(test3.state) - set(train3.state))\nprint(set(train3.state) - set(test3.state))\nprint(set(train3.state) & set(test3.state))\nstate_list0 = set(train3.state) & set(test3.state)\nstate_list = ['state_'+ x for x in state_list0 if str(x) != 'nan']\nstate_list\n"

In [41]:
#fine_amount_list0 = set(train3.fine_amount) & set(test3.fine_amount)
#fine_amount_list = ['fine_amount_'+ str(x) for x in fine_amount_list0 if str(x) != 'nan']

In [42]:
'''
print(agency_name_list)
print(disposition_list)
print(state_list)
common_list = agency_name_list + disposition_list + state_list + fine_amount_list
common_list
'''

'\nprint(agency_name_list)\nprint(disposition_list)\nprint(state_list)\ncommon_list = agency_name_list + disposition_list + state_list + fine_amount_list\ncommon_list\n'

In [43]:
'''
train3['fine_amount'] = train3.fine_amount.astype(str)
test3['fine_amount'] = test3.fine_amount.astype(str)
train_x = train3.iloc[:,:-1]
X = pd.get_dummies(train_x)[common_list].values
y = train3.iloc[:, -1].values
'''

"\ntrain3['fine_amount'] = train3.fine_amount.astype(str)\ntest3['fine_amount'] = test3.fine_amount.astype(str)\ntrain_x = train3.iloc[:,:-1]\nX = pd.get_dummies(train_x)[common_list].values\ny = train3.iloc[:, -1].values\n"

In [44]:
#print(X.shape)
#print(y.shape)

In [45]:
#X_test = pd.get_dummies(test3)[common_list].values
#X_test.shape

In [46]:
#from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(n_estimators = 501)
#clf.fit(X, y)

In [94]:
#result = clf.predict_proba(X_test)[:, 1]

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
def blight_model():
    
    #load data
    train = pd.read_csv('train.csv', encoding="latin-1")
    test = pd.read_csv('test.csv')
    
    #data selection
    col_list = ['ticket_id', 'agency_name', 'state','disposition','fine_amount', 'compliance'] #'ticket_issued_date', 'hearing_date', 'disposition', 'fine_amount'
    
    train1 = train[col_list]
    test1 = test[col_list[:-1]]
    train1 = train1[train1.compliance.notnull()]

    #set index
    train1.set_index('ticket_id', inplace=True)
    test1.set_index('ticket_id', inplace=True)
    
    #common features
    agency_name_list0 = set(train1.agency_name) & set(test1.agency_name)
    agency_name_list = ['agency_name_'+ x for x in agency_name_list0]
    
    disposition_list0 = set(train1.disposition) & set(test1.disposition)
    disposition_list = ['disposition_'+ x for x in disposition_list0]
    
    state_list0 = set(train1.state) & set(test1.state)
    state_list = ['state_'+ x for x in state_list0 if str(x) != 'nan']
      
    fine_amount_list0 = set(train1.fine_amount) & set(test1.fine_amount)
    fine_amount_list = ['fine_amount_'+ str(x) for x in fine_amount_list0 if str(x) != 'nan']
    
    common_list = agency_name_list + disposition_list + state_list + fine_amount_list
    
    #convert float to string
    train1['fine_amount'] = train1.fine_amount.astype(str)
    test1['fine_amount'] = test1.fine_amount.astype(str)
    
    #training dataset
    train_x = train1.iloc[:,:-1]
    X = pd.get_dummies(train_x)[common_list].values
    y = train1.iloc[:, -1].values
    
    #test dataset
    X_test = pd.get_dummies(test1)[common_list].values

    clf = RandomForestClassifier(n_estimators = 501, random_state=0)
    clf.fit(X, y)
    
    y_pred = clf.predict_proba(X_test)[:, 1]
    
    return pd.Series(y_pred, index=test1.index, dtype=np.float32)

In [49]:
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ticket_id
284932    0.056316
285362    0.011469
285361    0.071690
285338    0.056316
285346    0.071690
285345    0.056316
285347    0.061345
285342    0.394963
285530    0.000000
284989    0.051297
285344    0.061345
285343    0.011469
285340    0.011469
285341    0.061345
285349    0.076651
285348    0.040936
284991    0.000000
285532    0.023011
285406    0.023011
285001    0.012586
285006    0.000000
285405    0.016184
285337    0.042316
285496    0.061345
285497    0.056316
285378    0.016184
285589    0.054207
285585    0.056316
285501    0.071690
285581    0.011469
            ...   
376367    0.026435
376366    0.033070
376362    0.277469
376363    0.497224
376365    0.026435
376364    0.033070
376228    0.033070
376265    0.033070
376286    0.313667
376320    0.033070
376314    0.033070
376327    0.313667
376385    0.313667
376435    0.079298
376370    0.277469
376434    0.075066
376459    0.053724
376478    0.000000
376473    0.033070
376484    0.015299
376482    0.001733
37